In [ ]:
%load_ext tclmagic

#  If the prev line doesnt work  install it using the next line in your unix terminal
#  /depot/Python/Python-3.8.0/bin/pip install -U tcl-magic
#  /depot/Python/Python-3.8.0/bin/pip install -U notebook
#  You can configure it to ~/.local/ 

In [ ]:
%%tcl 

set SHELLTOOL_LOC "/remote/cad-rep/msip/tools/Shelltools"
lappend auto_path "$SHELLTOOL_LOC/ddr-utils-lay/dev/lib/tcl"
lappend auto_path "/depot/tcl8.6.3/lib"

set RealBin [file dirname [file normalize [info script]] ]
set RealScript [file tail [file normalize [info script]] ]
set PROGRAM_NAME $RealScript
set LOGFILE "[pwd]/log-$PROGRAM_NAME.log"

package require Messaging 1.0
namespace import ::Messaging::*
package require Misc 1.0
namespace import ::Misc::*

package require cmdline
package require try
package require yaml

# AI

- convert releaseMacro to a dictionary
- convert lists in lists...


In [ ]:
%%tcl
set sample_path "/u/alvaro/GitLab/my-personal-projects/experiments/tclyaml_experiment/inputs/sample.tcl"

In [ ]:
%%tcl

unset (

In [ ]:
%%tcl

proc create_dict_from_file {__file} {

    source $__file
    unset __file

    set varlist [lsort [info vars]]

    foreach varname $varlist {
        iprint $varname
        if {[regexp -all {\{} $varname match]} {
            set newvar $varname
            set newvar [regsub -all {\{} $newvar "\("]
            set newvar [regsub -all {\}} $newvar "\)"]
            set $newvar [set $varname]
            unset varname
        }
    }

    unset varlist
    set varlist [lsort [info vars]]

    foreach varname $varlist {
        iprint $varname
        if {[array exists $varname]} {
            foreach {ark arv} [array get $varname] {
                iprint "$varname $ark --> $arv"
                dict set legal_release $varname "$ark: " "\{$arv\}"
            }
        } else {
            if {[llength [set $varname]]>1} {
                if {![regexp {(date|prune|layers)} $varname match]} {
                    foreach litvalue [set $varname] {
                        lappend varvalue "- $litvalue"  
                    }
                } else {
                    set varvalue [set $varname]
                }
            } else {
                set varvalue [set $varname]
            }
            
            dict set legal_release $varname $varvalue
            unset varname
            unset varvalue
        }
        
        
    }
    return $legal_release
}

proc save_cleaned_yml_to_file {sample_path {file_name "yml-cleaned.yml"}} {

    set legal_release [create_dict_from_file $sample_path]
    set yaml_format [yaml::dict2yaml $legal_release 4 100]

    # Remove ": } {" to only colon
    set cleaned_format [regsub -all {\: \} \{} $yaml_format ": "]
    set cleaned_format [regsub -all {\} \{| \|-\n    \{|>\n    } $cleaned_format "\n    " ]
    
    set cleaned_values [regsub -all {\: \{} $cleaned_format ": " ]
    iprint $cleaned_values
    write_file $file_name [regsub -all {\}\n} [regsub -all {\}\n} $cleaned_values "\n" ] "\n" ]
 
}

proc save_raw_yml_to_file {sample_path {file_name "yml-raw.yml"}} {

    set legal_release [create_dict_from_file $sample_path]
    set yaml_format [yaml::dict2yaml $legal_release 4 100]

    write_file $file_name $yaml_format
 
}


In [ ]:
%%tcl
save_cleaned_yml_to_file $sample_path
save_raw_yml_to_file $sample_path